In [2]:
import pickle

In [3]:
import pickle
import re
import PyPDF2
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer
import nltk
from nltk.corpus import stopwords
from snowballstemmer import TurkishStemmer


In [4]:
with open('articles_2700.pickle', 'rb') as f:
    queries = pickle.load(f)

In [5]:
queries

[{'ArticleTitle': ['Türkiye’de Olağanüstü Hal: Devlet Aklı, Askerler ve Siviller'],
  'Author': ['ÖZTAN, Güven Gürkan\n                        ',
   'BEZCİ, Egemen B.\n                        '],
  'Publisher': ['Mülkiyeliler Birliği Genel Merkezi',
   'Mülkiye Alumni Association'],
  'Date': ['2015-04-10'],
  'DergiParkLink': ['https://dergipark.org.tr/tr/pub/mulkiye/issue/277/1251',
   None],
  'PDFLink': ['https://dergipark.org.tr/tr/download/article-file/1073'],
  'References': ['Agamben G (1998). Homo Sacer: Sovereign Power and Bare Life. California: Stanford University Press.',
   'Agamben G (2005). State of Exception. Chicago: University of Chicago Press.',
   'Aydınlı E, Özcan N A ve Akyaz D (2006). The Turkish Military’s March Toward Europe. Foreign Affairs, 85, 77-90.',
   'Benjamin W (2010). Şiddetin Eleştirisi Üzerine. İçinde: çev. E Göztepe, A Çelebi (der), Şiddetin Eleştirisi Üzerine, İstanbul: Metis, 19-43.',
   'Bezci E B (2014). Intelligence Reform and Threat Perceptio

In [6]:
len(queries[0]["References"])

55

In [7]:
title = queries[0]["ArticleTitle"]
title = ''.join(title).replace('\n', ' ').strip()

author = queries[0]["Author"]
author = ''.join(author).replace('\n', ' ').strip()

In [8]:
year = queries[0]["Date"]

In [9]:
year = ''.join(year)[:4]

In [10]:
year

'2015'

In [11]:
import re
import PyPDF2

In [12]:
def pdf_to_text(pdf_file_path):
    text = ""
    with open(pdf_file_path, 'rb') as pdf_file:
        pdf_reader = PyPDF2.PdfReader(pdf_file)
        for page_num in range(len(pdf_reader.pages)):
            page = pdf_reader.pages[page_num]
            text += page.extract_text()
    text = text.replace('\n', ' ')
    return text

In [13]:
text = pdf_to_text("000.pdf")

In [14]:
text

'159 Öztan G G ve Bezci E B (2015). Türkiye’de Olağanüstü Hal: Devlet Aklı, Askerler ve Siviller.  Mülkiye Dergisi , 39(1), 159-186.Türkiye’de Olağanüstü Hal:   Devlet Aklı, Askerler ve Siviller Güven Gürkan Öztan , İstanbul Üniversitesi Siyasal Bilgiler Fakültesi,  e-posta: guven.oztan@istanbul.edu.tr. Egemen B. Bezci, Nottingham Üniversitesi Tarih Bölümü,   e-posta: ahxebbe@nottingham.ac.uk. Özet 12 Eylül 1980 askeri darbesi Türkiye’de sosyal, ekonomik ve siyasal alanda geniş çaplı  değişikliklere yol açmıştır. Aynı zamanda darbe yönetimi, 1982 yılında yürürlü ğe giren  ve günümüzde de geçerliliğini sürdüren Anayasa’yı hazırlamıştır.  1982 anayasanın  önemli özelliklerinden biri olağanüstü hal kavramını Türkiye’nin siyasi arenasına  taşıması olmuştur. 1982 Anayasası ile birlikte anayasal düzene dâhil olan olağanüstü  hal, mevcut sıkıyönetim uygulamalarının yerine geçerek 1 Mart 1984 tarihinden  başlayarak uygulamadan kalktığı 30 Kasım 2002 tarihine kadar varlığını sürdürmüştür.  Bu ç

In [15]:
pattern_to_remove =  r'\b(\d+)\s+([\w\s]+)\((\d{4})\)\.\s*([^\.]+)\.\s*([^,]+),\s*(\d+)\s*\((\d+)\),\s*(\d+)-(\d+)\.\b'
matches = re.finditer(pattern_to_remove, text)

# Initialize a count variable
count = 0

# Iterate through the matches and count them
for match in matches:
    count += 1
    matched_sentence = match.group()
    print(matched_sentence)
    print("\n--------------\n")

# Print the total count of matching sentences
print(f"Total matches found: {count}")

159 Öztan G G ve Bezci E B (2015). Türkiye’de Olağanüstü Hal: Devlet Aklı, Askerler ve Siviller.  Mülkiye Dergisi , 39(1), 159-186.

--------------

160 Öztan G G ve Bezci E B (2015). Türkiye’de Olağanüstü Hal: Devlet Aklı, Askerler ve Siviller.  Mülkiye Dergisi , 39(1), 159-186.

--------------

161 Öztan G G ve Bezci E B (2015). Türkiye’de Olağanüstü Hal: Devlet Aklı, Askerler ve Siviller.  Mülkiye Dergisi , 39(1), 159-186.

--------------

162 Öztan G G ve Bezci E B (2015). Türkiye’de Olağanüstü Hal: Devlet Aklı, Askerler ve Siviller.  Mülkiye Dergisi , 39(1), 159-186.

--------------

163 Öztan G G ve Bezci E B (2015). Türkiye’de Olağanüstü Hal: Devlet Aklı, Askerler ve Siviller.  Mülkiye Dergisi , 39(1), 159-186.

--------------

164 Öztan G G ve Bezci E B (2015). Türkiye’de Olağanüstü Hal: Devlet Aklı, Askerler ve Siviller.  Mülkiye Dergisi , 39(1), 159-186.

--------------

165 Öztan G G ve Bezci E B (2015). Türkiye’de Olağanüstü Hal: Devlet Aklı, Askerler ve Siviller.  Mülkiye 

In [16]:
text = re.sub(pattern_to_remove, '', text)
text

'Türkiye’de Olağanüstü Hal:   Devlet Aklı, Askerler ve Siviller Güven Gürkan Öztan , İstanbul Üniversitesi Siyasal Bilgiler Fakültesi,  e-posta: guven.oztan@istanbul.edu.tr. Egemen B. Bezci, Nottingham Üniversitesi Tarih Bölümü,   e-posta: ahxebbe@nottingham.ac.uk. Özet 12 Eylül 1980 askeri darbesi Türkiye’de sosyal, ekonomik ve siyasal alanda geniş çaplı  değişikliklere yol açmıştır. Aynı zamanda darbe yönetimi, 1982 yılında yürürlü ğe giren  ve günümüzde de geçerliliğini sürdüren Anayasa’yı hazırlamıştır.  1982 anayasanın  önemli özelliklerinden biri olağanüstü hal kavramını Türkiye’nin siyasi arenasına  taşıması olmuştur. 1982 Anayasası ile birlikte anayasal düzene dâhil olan olağanüstü  hal, mevcut sıkıyönetim uygulamalarının yerine geçerek 1 Mart 1984 tarihinden  başlayarak uygulamadan kalktığı 30 Kasım 2002 tarihine kadar varlığını sürdürmüştür.  Bu çalışma, olağanüstü hal uygulamasının devlet aklının bir tezahürü olduğunu iddia  etmektedir. Tehdit algılamalarına göre şekillenen 

In [17]:
def split_article(article_text):
    reference_pattern = r"Kaynakça|References|Bibliography|Citations|Sonnotlar"
    reference_match = re.search(reference_pattern, article_text, re.IGNORECASE)

    if reference_match:
        main_text = article_text[:reference_match.start()] #extract main text from references
        reference_section = article_text[reference_match.start():]

        return main_text, reference_section
    else:
        return article_text, None

text, references = split_article(text)

In [18]:
def window(text, loc, target_str, left_window_size=150, right_window_size=100):
    left_threshold = max(loc - left_window_size, 0)
    right_threshold = min(loc + right_window_size, len(text))

    return text[left_threshold:right_threshold]


def find_references_with_context(text, title, author, year):
    pattern = re.compile(r'\(.*?\d{4}.*?\)')
    references_with_context = []

    for match in pattern.finditer(text):
        if match.group().lower() != title.lower() and author not in match.group() and str(year) not in match.group():
            match_text = match.group()
            start_loc = text.find(match_text)
            context = window(text, start_loc, match_text, 150, 100)
            references_with_context.append(context)

    return references_with_context

In [19]:
sentences_with_references=find_references_with_context(text, title, author, year)

In [20]:
sentences_with_references

['y in Turkey and derive  the necessary empirical argument in the theoretical discussion, this article analyses  minutes of the National Security Board (Milli Güvenlik Konseyi) and Consultation  Assembly (Danışma Meclisi) during the relevant discussion',
 'incelenmesi bağlamında aynı saptamayı yapmak zordur; olağanüstü  hal rejimini odağına yerleştiren araştırmaların sayısı ve kapsamı görece sınırlıdır  (Gemalmaz, 1991; Üskül, 1988; Duran, 1995).  1980 ile 1983 yılları arasındaki  süreçte ülkeyi doğrud',
 ' Üskül, 1988; Duran, 1995).  1980 ile 1983 yılları arasındaki  süreçte ülkeyi doğrudan yöneten askeri rejim, orta vadedeki çabalarını devlet  aklının (raison d’état) çerçevesini çizdiği yeni bir anayasa hazırlatmak üzerine yoğunlaştırmıştır. Yeni ana',
 'ile silahlı mücadelesinde devlet otoritesini  yeniden tesis etmek adına olağanüstü hal rejiminden yararlandığını ve askerlerin  Milli Güvenlik Kurulu (MGK) üzerinden olağanüstü hale karar veren esas yetkili  merci ve siyasal özne old

In [21]:
for i in sentences_with_references:
    print(i)
    print("--------")

y in Turkey and derive  the necessary empirical argument in the theoretical discussion, this article analyses  minutes of the National Security Board (Milli Güvenlik Konseyi) and Consultation  Assembly (Danışma Meclisi) during the relevant discussion
--------
incelenmesi bağlamında aynı saptamayı yapmak zordur; olağanüstü  hal rejimini odağına yerleştiren araştırmaların sayısı ve kapsamı görece sınırlıdır  (Gemalmaz, 1991; Üskül, 1988; Duran, 1995).  1980 ile 1983 yılları arasındaki  süreçte ülkeyi doğrud
--------
 Üskül, 1988; Duran, 1995).  1980 ile 1983 yılları arasındaki  süreçte ülkeyi doğrudan yöneten askeri rejim, orta vadedeki çabalarını devlet  aklının (raison d’état) çerçevesini çizdiği yeni bir anayasa hazırlatmak üzerine yoğunlaştırmıştır. Yeni ana
--------
ile silahlı mücadelesinde devlet otoritesini  yeniden tesis etmek adına olağanüstü hal rejiminden yararlandığını ve askerlerin  Milli Güvenlik Kurulu (MGK) üzerinden olağanüstü hale karar veren esas yetkili  merci ve siy

In [22]:
len(sentences_with_references)

53

In [23]:
sentences_with_references = list(set(sentences_with_references))

In [24]:
references = queries[0]["References"]

In [25]:
len(references)

55

In [26]:
references

['Agamben G (1998). Homo Sacer: Sovereign Power and Bare Life. California: Stanford University Press.',
 'Agamben G (2005). State of Exception. Chicago: University of Chicago Press.',
 'Aydınlı E, Özcan N A ve Akyaz D (2006). The Turkish Military’s March Toward Europe. Foreign Affairs, 85, 77-90.',
 'Benjamin W (2010). Şiddetin Eleştirisi Üzerine. İçinde: çev. E Göztepe, A Çelebi (der), Şiddetin Eleştirisi Üzerine, İstanbul: Metis, 19-43.',
 'Bezci E B (2014). Intelligence Reform and Threat Perception in Turkey. Journal of Mediterranean and Balkan Intelligence, 3, 25-42.',
 'Bilgiç T Ü (2009). The Military and Europenization Reforms in Turkey. Middle Eastern Studies, 45, 803-824.',
 'Bozaslan H (2014). “Neden Silahlı Mücadele?”: Türkiye Kürdistanı’nda Şiddeti Anlamak. İçinde: Çeğin G ve Şirin İ (der), Türkiye’de Siyasal Şiddetin Boyutları. İstanbul: İletişim Yayınları, 149-164.',
 'Brown J (1989). The Military and Society: The Turkish Case. Middle Eastern Studies, 25, 387-404.',
 'Cizr

In [27]:
sentences_with_references

['i çözmede  salt sivil siyasete ve topluma güvenilmeyeceğini düşünmüş ve uzunca bir süre  cumhuriyetin ve demokrasinin patentini kendine ait görmüştür (Bozdemir,  1982: 163). Bu nedenle, Silahlı Kuvvetler, MGK üzerinden vatan ülküsünde  vücut bulmuş k',
 'tlerinin meşru müdafaa durumuna düşmeleri halinde görevli güvenlik kuvvetleri mensupları doğruca ve duraksamadan hedefe ateş etme  yetkisi” vermiştir (Danışma Meclisi, 1983: 7). Devletin şiddet tekelinin kırıldığı  anlarda ve alanlarda Silahlı Kuvvet',
 'n illerde, valilerin ihtiyaç görmesi halinde civar illerden ve askeri birliklerden hangi koşullar altında destek kuvvet isteyeceğini düzenlemektedir  (Danışma Meclisi, 1983: 3). Komisyon’un önerdiği tasarı, valilerin sadece son  çare olarak askeri bi',
 'de yetkiler verilmiştir.  Bundan sonra doğabilecek hadiseler; eğer doğar ise, idare amirlerimizin  aczinden ileri gelecektir, hukuk boşluğundan değil (Danışma Meclisi, 1983:  33). Gürel tasarı üzerine lehte konuşmasında, Olağanüstü H

In [28]:
def extract_citations(window):
    pattern = re.compile(r'\(.*?\d{4}.*?\)')
    return pattern.findall(window)

def match_reference(citation, references):
    # Extract year and try to find a part of the author's name
    match = re.search(r'(\d{4})', citation)
    if match:
        year = match.group(1)
        # Split citation and look for potential author name parts
        for part in re.split(r'\W+', citation):
            if part and len(part) > 2 and part.lower() != year:  # Exclude very short parts and the year
                for ref in references:
                    if year in ref and part in ref:
                        return ref
    return "No matching reference found"

In [29]:
for window in sentences_with_references:
    citations = extract_citations(window)
    for citation in citations:
        matched_reference = match_reference(citation, references)
        print(f"Window: {window}")
        print(f"Citation: {citation}")
        print(f"Matched Reference: {matched_reference}\n")

Window: i çözmede  salt sivil siyasete ve topluma güvenilmeyeceğini düşünmüş ve uzunca bir süre  cumhuriyetin ve demokrasinin patentini kendine ait görmüştür (Bozdemir,  1982: 163). Bu nedenle, Silahlı Kuvvetler, MGK üzerinden vatan ülküsünde  vücut bulmuş k
Citation: (Bozdemir,  1982: 163)
Matched Reference: No matching reference found

Window: tlerinin meşru müdafaa durumuna düşmeleri halinde görevli güvenlik kuvvetleri mensupları doğruca ve duraksamadan hedefe ateş etme  yetkisi” vermiştir (Danışma Meclisi, 1983: 7). Devletin şiddet tekelinin kırıldığı  anlarda ve alanlarda Silahlı Kuvvet
Citation: (Danışma Meclisi, 1983: 7)
Matched Reference: Danışma Meclisi (1983). Olağanüstü Hal Kanun Tasarısı. Danışma Meclisi Tutanak Dergisi. Birleşim: 153, Oturum: 1, 29.08.1983

Window: n illerde, valilerin ihtiyaç görmesi halinde civar illerden ve askeri birliklerden hangi koşullar altında destek kuvvet isteyeceğini düzenlemektedir  (Danışma Meclisi, 1983: 3). Komisyon’un önerdiği tasarı, vali

In [30]:
citations_2 = []

for window in sentences_with_references:
    citations = extract_citations(window)
    for citation in citations:
        citations_2.append(citation)

In [31]:
def extract_citations(window):
    pattern = re.compile(r'\(.*?\d{4}.*?\)')
    return pattern.findall(window)

In [32]:
def calculate_cosine_similarity_tfidf(citations, references):
    # Combine both lists of strings
    all_strings = citations + references

    # Create a TF-IDF vectorizer
    vectorizer = TfidfVectorizer()

    # Transform the strings into TF-IDF vectors
    tfidf_matrix = vectorizer.fit_transform(all_strings)

    # Calculate cosine similarity between each pair of strings
    similarity_matrix = cosine_similarity(tfidf_matrix)

    # Iterate through citations and find the best match in references
    for i, citation in enumerate(citations):
        # Find the index of the most similar reference
        best_match_index = similarity_matrix[i, len(citations):].argmax()

        # Get the corresponding reference string
        best_match_reference = references[best_match_index]

        # Get the cosine similarity score
        similarity_score = similarity_matrix[i, len(citations) + best_match_index]

        # Print the results
        if similarity_score > 0.25:
            print(f"Citation: {citation}")
            print(f"Reference: {best_match_reference}")
            print(f"Cosine Similarity Score: {similarity_score:.4f}")
            print("-" * 30)

calculate_cosine_similarity_tfidf(citations_2, references)

Citation: (Danışma Meclisi, 1983: 7)
Reference: Danışma Meclisi (1983). Olağanüstü Hal Kanun Tasarısı. Danışma Meclisi Tutanak Dergisi. Birleşim: 153, Oturum: 1, 29.08.1983
Cosine Similarity Score: 0.6482
------------------------------
Citation: (Danışma Meclisi, 1983: 3)
Reference: Danışma Meclisi (1983). Olağanüstü Hal Kanun Tasarısı. Danışma Meclisi Tutanak Dergisi. Birleşim: 153, Oturum: 1, 29.08.1983
Cosine Similarity Score: 0.6482
------------------------------
Citation: (Danışma Meclisi, 1983:  33)
Reference: Danışma Meclisi (1983). Olağanüstü Hal Kanun Tasarısı. Danışma Meclisi Tutanak Dergisi. Birleşim: 153, Oturum: 1, 29.08.1983
Cosine Similarity Score: 0.4965
------------------------------
Citation: (Cizre, 1993: 277)
Reference: Serxwebun (10/1993).
Cosine Similarity Score: 0.2754
------------------------------
Citation: (Milliyet, 27/06/1995: 15)
Reference: Milliyet (27/06/1995).
Cosine Similarity Score: 0.8938
------------------------------
Citation: (Milliyet, 30/07/1996:

In [33]:
citations_2

['(Bozdemir,  1982: 163)',
 '(Danışma Meclisi, 1983: 7)',
 '(Danışma Meclisi, 1983: 3)',
 '(Danışma Meclisi, 1983:  33)',
 '(Cizre, 1993: 277)',
 '(bu tür kararların kaldırılabilmesi için yeniden sıkıyönetim  ilanını istemek gerekiyor!) (1988: 89)',
 '(Milliyet, 27/06/1995: 15)',
 '(Milliyet, 30/07/1996: 16)',
 '(2000: 116)',
 '(Danışma Meclisi, 1983: 6)',
 '(Yüksekova Haber, 14/08/2011)',
 '(Bezci, 2014: 29-34)',
 '(Milliyet, 03/10/1984: 6)',
 '(1999: 31)',
 '(1961: 39-54)',
 '(Üskül, 1988: 103)',
 '(Özcan, 2007: 43)',
 '(Milliyet,  25/08/1984: 6)',
 '(Harp Akademileri Komutanlığı, 1993: 20)',
 '(Aydınlı, Özcan ve Akyaz, 2006: 78)',
 '(2014: 221)',
 '(07/10/1984: 7)',
 '(Danışma Meclisi, 1983: 5)',
 '(1984)',
 '(İnsel, 1997:  15-19)',
 '(2005: 19-20)',
 '(Milliyet, 09/04/1993:  13)',
 '(2014: 158)',
 '(Milliyet, 25/01/1997: 7)',
 '(1984: 2)',
 '(Gemalmaz, 1991; Üskül, 1988; Duran, 1995)',
 '(2004: 143)',
 '(2007: 15)',
 '(2005: 647)',
 '(2000: 95)',
 '(Öztan, 2014: 192)',
 '(Cizre, 19

In [34]:
len(references)

55

In [69]:
def extract_titles(references):
    titles = []
    for ref in references:
        match = re.search(r'\)\. (.+?)\.', ref)
        if match:
            titles.append(match.group(1).lower())
        else:
            match = re.search(r'\. (.+?)\. [A-Z][a-z]+', ref)
            if match:
                titles.append(match.group(1).lower())
    return titles

titles = extract_titles(references)
for title in titles:
    print(title)


homo sacer: sovereign power and bare life
state of exception
the turkish military’s march toward europe
şiddetin eleştirisi üzerine
intelligence reform and threat perception in turkey
the military and europenization reforms in turkey
“neden silahlı mücadele?”: türkiye kürdistanı’nda şiddeti anlamak
the military and society: the turkish case
ap-ordu i̇lişkileri: bir i̇kilemin anatomisi
muktedirlerin siyaseti: merkez sağ-ordu-i̇slamcılık
democratic transition in turkey, 1980–83: the impact of european diplomacy
olağanüstü hal kanun tasarısı
olağanüstü hal kanun tasarısı
lessons of military regimes and democracy: the turkish case in a comparative perspective
bunalım kararnameleri
devlet ve politik şiddet: latin amerika ve türkiye’de gerilla hareketleri
security, territory, population: lectures at the college de france 1977-1978
historical roots of martial law within the turkish legal system: perspectives and text
güneydoğu anadolu’da devam etmekte olan bölücü hareketin gelecekteki muhteme

In [70]:
references

['Agamben G (1998). Homo Sacer: Sovereign Power and Bare Life. California: Stanford University Press.',
 'Agamben G (2005). State of Exception. Chicago: University of Chicago Press.',
 'Aydınlı E, Özcan N A ve Akyaz D (2006). The Turkish Military’s March Toward Europe. Foreign Affairs, 85, 77-90.',
 'Benjamin W (2010). Şiddetin Eleştirisi Üzerine. İçinde: çev. E Göztepe, A Çelebi (der), Şiddetin Eleştirisi Üzerine, İstanbul: Metis, 19-43.',
 'Bezci E B (2014). Intelligence Reform and Threat Perception in Turkey. Journal of Mediterranean and Balkan Intelligence, 3, 25-42.',
 'Bilgiç T Ü (2009). The Military and Europenization Reforms in Turkey. Middle Eastern Studies, 45, 803-824.',
 'Bozaslan H (2014). “Neden Silahlı Mücadele?”: Türkiye Kürdistanı’nda Şiddeti Anlamak. İçinde: Çeğin G ve Şirin İ (der), Türkiye’de Siyasal Şiddetin Boyutları. İstanbul: İletişim Yayınları, 149-164.',
 'Brown J (1989). The Military and Society: The Turkish Case. Middle Eastern Studies, 25, 387-404.',
 'Cizr

In [71]:
print(len(titles))

39


In [38]:
"Türkiye’de Olağanüstü Hal: Devlet Aklı, Askerler ve Siviller" in queries[0]['ArticleTitle']

True

In [39]:
articles_with_lowercase_titles = [
    {**article, 'ArticleTitle': [title.lower() for title in article.get('ArticleTitle', [])]}
    for article in queries
]

In [40]:
target_title = "Alkol-Madde Kullanıcılarının Bağımlılık Şiddeti İle Aile Üyelerinin Eş Bağımlılık Düzeyleri Arasındaki İlişki".lower()

In [41]:
matching_queries = [query for query in articles_with_lowercase_titles if 'ArticleTitle' in query and target_title in query['ArticleTitle']]

In [42]:
matching_queries

[{'ArticleTitle': ['alkol-madde kullanıcılarının bağımlılık şiddeti i̇le aile üyelerinin eş bağımlılık düzeyleri arasındaki i̇lişki',
   'relationship between addiction severity of alcohol-substance users and co-dependency levels of family members'],
  'Author': ['KAPLAN, Özgen\n                        ',
   'ÖZBARAN, Burcu\n                        '],
  'Publisher': ['Galenos Yayınevi', 'Galenos Yayınevi'],
  'Date': ['2023-07-03'],
  'DergiParkLink': ['https://dergipark.org.tr/tr/pub/bagimli/issue/80424/1322007',
   '10.51982/bagimli.1322007'],
  'PDFLink': ['https://dergipark.org.tr/tr/download/article-file/3240770'],
  'References': ['Tot Ş, Yazıcı K, Yazıcı A, et al. Psychosocial correlates of subs-tance use among adolescents in Mersin, Turkey. Public Health 2004; 118(8): 588-593',
   'Saunders EC, McLeman BM, McGovern MP, et al. The influence of family and social problems on treatment outcomes of persons with cooccurring substance use disorders and PTSD. J Subst Use 2016; 21(3): 

In [43]:
temp_ref = matching_queries[0]["References"]

In [44]:
temp_ref

['Tot Ş, Yazıcı K, Yazıcı A, et al. Psychosocial correlates of subs-tance use among adolescents in Mersin, Turkey. Public Health 2004; 118(8): 588-593',
 'Saunders EC, McLeman BM, McGovern MP, et al. The influence of family and social problems on treatment outcomes of persons with cooccurring substance use disorders and PTSD. J Subst Use 2016; 21(3): 237-243',
 'Ögel K, Çelikay H, Başabak A. Aile bağımlılık profil indeksinin geliştirilmesi, geçerlik ve güvenirliği. Klinik Psikiyatri Dergisi 2017; 20(4): 255-262.',
 'Aka P, Özok Hİ. Eş Bağımlılık. Tanhan F (editör). Psikolojik Danışmada Güncel Konular. Ankara, 2022; 39-45.',
 'Lancer D. Conquering Shame and Codependency: 8 Steps To Freeing The True You. Minnesota: Hazelden Publishing, 2014; 1-23.',
 'Ançel G. Karşılıklı bağımlılık kavramı: hemşirelikle ilişkisi ve karşılıklı bağımlılığı belirleme araçları. Hemşirelikte Araştırma Geliştirme Dergisi 2012; 14(1): 70-78.',
 'Zerwekh J, Michaels B. Co-dependency: assessment and recovery. Nur

In [72]:
titles = extract_titles(temp_ref)

In [73]:
titles

['psychosocial correlates of subs-tance use among adolescents in mersin, turkey',
 'aile bağımlılık profil indeksinin geliştirilmesi, geçerlik ve güvenirliği',
 'psikolojik danışmada güncel konular',
 'conquering shame and codependency: 8 steps to freeing the true you',
 'karşılıklı bağımlılık kavramı: hemşirelikle ilişkisi ve karşılıklı bağımlılığı belirleme araçları',
 'co-dependency: assessment and recovery',
 'et al, the questionnaire of family functioning: a preliminary validation of a standardized instrument to evaluate psychoeducational family treatments',
 'co-dependency assessment inventory: a preliminary research tool',
 'a critique of the codependency concept considering the best interests of the child',
 'co‐dependency: a critical review',
 'choice making: for co-dependents, adult children and spirituality seekers',
 'codependency: an empirical study from a systemic perspective',
 'family stressors as predictors of codependency',
 'codependency explored: a social movement i

In [74]:
matched = [query for query in articles_with_lowercase_titles if 'ArticleTitle' in query and any(ref in query['ArticleTitle'] for ref in titles)]

In [75]:
matched

[{'ArticleTitle': ['bağımlılıkta eş bağımlılık faktörü (beşf) ölçeği’nin geçerlik ve güvenilirlik çalışması',
   'validity and reliability of co-dependency in substance use disorder scale (codsuds)'],
  'Author': ['ŞİMŞEK, Melike\n                        ',
   'ÖNCÜ, Fatihcan\n                        ',
   'KABİL, Melekcan\n                        '],
  'Publisher': ['Lut TAMAM', 'Lut TAMAM'],
  'Date': ['2020-09-27'],
  'DergiParkLink': ['https://dergipark.org.tr/tr/pub/pgy/issue/48355/800752',
   '10.18863/pgy.800752'],
  'PDFLink': ['https://dergipark.org.tr/tr/download/article-file/1312168']},
 {'ArticleTitle': ['üniversite öğrencilerinde madde kullanımı, şiddet ve bazı psikolojik özellikler',
   'üniversite öğrencilerinde madde kullanımı, şiddet ve bazı psikolojik  özellikler'],
  'Author': ['TURHAN, Ebru\n                        ',
   'İNANDI, Tacettin\n                        ',
   'ÖZER, Cahit\n                        ',
   'AKOĞLU, Sabahat\n                        '],
  'Publi

In [55]:
extract_titles(queries[456]["References"])

['conservative principles for environmental reform',
 'environment and social theory, second edition, london: routledge',
 'muhafazakârlık',
 'the conservative case for the environment',
 'traditionalist conservatism and environmental ethics',
 'hegemonya ve rıza: politik bir strateji',
 'green political thought, fourth edition, london: routledge',
 'politics and environment in america: partisan and ideological cleavages in public support for environmentalism',
 'donald trump has tweeted climate change skepticism 115 times',
 'environmentalism and political theory: toward an ecocentric approach',
 'free market environmentalism: friend or foe? environmental politics, 2(1), 1-19',
 'the conservative as environmentalist: from goldwater and the early reagan to 21st century',
 'the greening of the tories: the conservative party and the environment',
 'the history of european conservative thought',
 'beyond the new right: markets, government and the common environment',
 'straw dogs: thought

In [59]:
queries[40567]["References"]

['Becktor JP, Einersen S, Kjaer I. A sella turcica bridge in subjects with severe craniofacial deviations. Eur J Orthod 2000;22:69-74.',
 'Camp JD. The normal and pathologic anatomy of the sella turcica as revealed at necropsy. Radiology 1923;1:65-73.',
 'Bergland RM, Ray BS, Torack RM. Anatomical variations in the pituitary gland and adjacent structures in 225 human autopsy cases. J Neurosurg 1968;28:93-99.',
 'Carstens M. Die selladiagnostik. Fortschr Geb Rontgenstrahlen 1949;71:257-272.',
 'Jones RM, Faqir A, Millett DT, Moos KF, McHugh S. Bridging And Dimensions Of Sella Turcica in Subjects Treated By Surgical-Orthodontic Means or Orthodontics Only. Eur J Orthod 2005;75:714-718.',
 'Taveras JM, Wood EH. Diagnostic Neuroradiology. MD: Williams & Wilkins Co 1964.',
 'Björk A. Cranial base development: A follow-up x- ray study of the individual variation in growth occurring between the ages of 12 and 20 years and its relation to brain case and face development . Am J Orthod 1955;41:19